In [18]:
import time
from bs4 import BeautifulSoup
import csv
import selenium 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

service = Service(ChromeDriverManager().install())

In [16]:
# File Paths
STORELIST = "storelist.xlsx"
EXPORT = "export.xlsx"
EXPORT_CSV = "export.csv"

In [3]:
specials_page = []

def getspecials(url, StoreID):
    #Launch Chrome
    driver = webdriver.Chrome(service=service)

    # Go To Store Main Menu
    #URL = "https://www.papajohns.com/order/viewStoreMenu/CARRYOUT/BUSINESS/190/"
    driver.get(url)
    time.sleep(1)

    #Close Cookies Window
    ccb = driver.find_element(By.XPATH, '//*[@id="onetrust-close-btn-container"]/button')
    ccb.click()
    time.sleep(1)

    #Close LTO Window
    #cjb = driver.find_element(By.XPATH, '//*[@id="pizza-img"]/div[2]/div/button')
    #cjb.click()
    #time.sleep(2)

    #Go To Specials Page
    spl = driver.find_element(By.XPATH, '/html/body/div[2]/header/section[2]/nav/ul/li[2]/a')
    spl.click()
    time.sleep(3)

    #Locate & Parse Specials HTML
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    cards = soup.find_all('div',{'class':'card-details'})

    #Extract Offers & Prices
    for card in cards:
        data_dict              = {}
        data_dict['store']     = StoreID
        data_dict['offer']     = card.find('h2').text
        data_dict['description'] = card.find('p',{'class':'description'}).text
        try:
            data_dict['price']     = card.find('span',{'class':'price'}).text
        except:
            data_dict['price']     = card.find('span',{'class':'price'})
        specials_page.append(data_dict)
    time.sleep(1)
    driver.close()

In [6]:
#Generate Store List
pathing = pd.read_excel(STORELIST)
pathing_list = pathing['Store#'].tolist()

In [7]:
for store in pathing_list:
    url = f"https://www.papajohns.com/order/viewStoreMenu/CARRYOUT/BUSINESS/" + str(store) +"/"
    StoreID = str(store)
    getspecials(url, StoreID)
    print(url)

https://www.papajohns.com/order/viewStoreMenu/CARRYOUT/BUSINESS/1740/
https://www.papajohns.com/order/viewStoreMenu/CARRYOUT/BUSINESS/1743/
https://www.papajohns.com/order/viewStoreMenu/CARRYOUT/BUSINESS/2172/
https://www.papajohns.com/order/viewStoreMenu/CARRYOUT/BUSINESS/2174/


In [27]:
with open(EXPORT_CSV, mode='w',newline='') as csv_file:
    headers = ['store','offer','description','price']
    writer = csv.DictWriter(csv_file, fieldnames=headers)
    writer.writeheader()
    writer.writerows(specials_page)
    
    


{'store': '1740', 'offer': 'Garlic Epic Stuffed Crust: A pizza inspired by you', 'description': 'Garlic sauce fans, you inspired us! We transformed your love for our garlic sauce into our pizza. Enjoy a 1-topping Garlic Epic Stuffed Crust pizza with garlicky goodness inside the cheesy crust, drizzled onto it, and served on the side.', 'price': ' $13.99 '}
{'store': '1740', 'offer': 'Calzone Papa Bites', 'description': ' Indulge in our Calzone Papa Bites. Packed with a blend of mozzarella and garlic herb ricotta, pepperoni, Italian sausage, and green peppers inside our fresh original dough. Served with original pizza sauce on the side for even more calzone flavor. ', 'price': '$ 4.99 '}
{'store': '1740', 'offer': 'Papa Pairings', 'description': ' Craving what you see? Our newest addition to your Papa Pairings possibilities: Wings! Get any of these and more delectable Papa Johns favorites for just $6.99 each when you pair two or more. ', 'price': '$ 6.99 '}
{'store': '1740', 'offer': 'La